Modify the paths and constants if needed. If sudo is not allowed, you need to set "libcuda_path" and "library_path" manually if they are not correct. 

Then run the cell below to load necessary methods and libraries

In [4]:
model_translate_path = "haoranxu/ALMA-13B-R"
model_summarize_path = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
libcuda_path = "/usr/local/cuda/compat"
library_path = "/usr/local/cuda/lib64"

USE_UNSLOTH=True #NOTE: Not using Unsloth affects the generated summary
SUDO=False
import os

if not SUDO:
    libcuda_path = "/usr/local/cuda/compat" #contains file libcuda.so
    library_path = "/usr/local/cuda/lib64" #contains files like libcublas.so, libcufile.so...

    os.environ["TRITON_LIBCUDA_PATH"]=libcuda_path
    os.environ["LIBRARY_PATH"]=library_path

from unsloth import FastLanguageModel
import json
import torch
import nltk
import re

nltk.download('punkt')

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")
print(torch.cuda.mem_get_info())


def load_model(model_path, max_seq_length): #
    max_seq_length = 8192 
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True
    
    if USE_UNSLOTH:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = model_summarize_path, # YOUR MODEL YOU USED FOR TRAINING
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
        )
        FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    else:
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_summarize_path, # YOUR MODEL YOU USED FOR TRAINING
            load_in_4bit = load_in_4bit,
        )
        tokenizer = AutoTokenizer.from_pretrained(mistral_tokenizer_path)
    return model, tokenizer

def alma_prompt(texts, src_lang="Czech", out_lang="English"):
    formatted_texts = texts.copy()
    for i, text in enumerate(texts):
        formatted_texts[i] = f"Translate this from {src_lang} to {out_lang}:\n{src_lang}: {text}\n{out_lang}:"
    return formatted_texts

def clean_text(text):
    return text.replace("-\n","").replace('\r', ' ').replace('\n', ' ')

def load_dataset(path):
    with open(path, 'r') as j:
         contents = json.loads(j.read())
    return contents

def chunk_sentences(text, n, lang):
    # Initialize an empty list to store the joined sentences
    sentence_split = nltk.sent_tokenize(text=text, language=lang)
    joined_sentences = []
    
    # Iterate over the sentences list with a step of n
    for i in range(0, len(sentence_split), n):
        # Join the current and the next n-1 sentences and append to the list
        joined_sentences.append(' '.join(sentence_split[i:i+n]))
    
    return joined_sentences

def translate_text(prompt, model, tokenizer, max_new_tokens=2048, temperature=1, top_p=1, repetition_penalty = 1.3, **kwargs):
    inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = max_new_tokens, **kwargs)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True, temperature=temperature, top_p=top_p, repetition_penalty = repetition_penalty)
    #print(decoded_outputs)
    return decoded_outputs[0].split(prompt)[1]

def translate_article(text, model, tokenizer, prompt_func, src_lang, out_lang, max_new_tokens=2048, chunk_size=10):
    text = clean_text(text)
    chunked_sentences = chunk_sentences(text, chunk_size, "czech")
    formatted_chunked_sentences = prompt_func(chunked_sentences, src_lang, out_lang)
    #print(formatted_chunked_sentences)
    translated_article = []
    for i, chunk_sentence in enumerate(formatted_chunked_sentences):
        #print(f"{i}/{len(formatted_chunked_sentences)}")
        translated_text = translate_text(chunk_sentence, model, tokenizer, max_new_tokens)
        translated_article.append(translated_text)
    return ''.join(translated_article)

                        

def remove_enumeration(text):
    pattern = r'^\s*\d+\.\s*'
    cleaned_lines = [re.sub(pattern, '', line) for line in text.split('\n')]
    return '\n'.join(cleaned_lines)


def summarize_text(prompt, model, tokenizer, max_new_tokens=512, temperature=0.3, top_p=1, **kwargs):
    #print(tokenizer.apply_chat_template(prompt, tokenize=False, return_tensors = "pt").to("cuda"))
    inputs = tokenizer.apply_chat_template(prompt, return_tensors = "pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens = max_new_tokens)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True, temperature=temperature, top_p=top_p, **kwargs)
    #print(decoded_outputs[0].split(prompt))
    split = decoded_outputs[0].split("[/INST]")
    return split[len(split)-1]

def mistral_prompt(text):
    text = f"Summarize the following text in five sentences: {text}"
    messages = [
        {"role": "user", "content": f"Summarize my texts using only 5 sentences"},
        {"role": "assistant", "content": f"Sure. I will write summaries in the style of a news reporter and use only 5 sentences."},
        {"role": "user", "content": f"{text}"},
    ]
    return messages

def tst(text, max_new_tokens=2048, chunk_size=10):
    alma_max_seq_len = 2048
    mistral_max_seq_len = 8196
    
    model, tokenizer = load_model(model_translate_path, max_seq_length=alma_max_seq_len)
    text_to_summarize = translate_article(text, model, tokenizer, alma_prompt, "Czech", "English", max_new_tokens=max_new_tokens, chunk_size=chunk_size)
    
    model, tokenizer = load_model(model_summarize_path, max_seq_length=mistral_max_seq_len)
    summarized_text = remove_enumeration(summarize_text(mistral_prompt(text_to_summarize), model, tokenizer, max_new_tokens))
    
    model, tokenizer = load_model(model_translate_path, max_seq_length=alma_max_seq_len)
    summary = translate_article(summarized_text, model, tokenizer, alma_prompt, "English", "Czech", max_new_tokens=max_new_tokens, chunk_size=chunk_size)
    
    return summary



Number of GPUs available: 1
(10429464576, 20937965568)


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
text = """Ročník XI.\nV Domažlicích, dne 2. září 1882.\nČíslo 36.\nPOSEL OD ČERCHOVA,\nDomažlický týdenník pro zájmy politické, obecné a spolkové.\nDopisy, inseráty a předplatné peníze buďtež zasílány redakci franko.\nReklamace nezapečetěné se nefrankují. Dopisy bez podpisu se nepřijímají, ruko-\nVychází každou sobotu o 4. hodině odpoledne, v knihkupectví Jiřího Prunara na\nČtvrtletně:\nnáměstí a stojí:\npisy se nevracejí. Plat inserátů obnáší za řádku třísloupcovou poprvé 3 kr., za každé\nPoštovní zásylkou:\nPro Domažlice:\nza hranice . . 1 zl. 33 kr.\nnásledující uveřejnění 2 kr. a platí se hned při objednání.\nčtvrtletně . . 1 zl. 20 kr.\nzl.\n1\ndo Ameriky . . 1 „ 46 „\nčtvrtletně . . . .\nJednotlivá čísla stojí 10 kr.\npůlletně . . 2 „ 40 „\n. . . . 2\npůlletně\nDonáška do domu 6 kr.\nročně . . . 4 „ 80 „\nročně . . . . . 4\nad\na\ns\náno\njed\nn\nrick\nm\nosle\nDle nové organisace armády rakouské bude\nstav pěchoty upraven nyní na 4 stupně a sice: Stav v\nmíru normální s 3 důstojníky a 83 muži pro jednot-\nlivou setninu; stav v míru snížený s 3 důstojníky a\n68 muži; stav v míru zvýšený s 3 důstojníky a 128\nmuži a konečně plný stav válečný s 4 důstojníky a\n232 muži po setnině. Stav válečný setniny doplňo-\nvací čítati bude 4 důstojníky a 232 muže. V stavu\npokojném obsahovati bude jedenkaždý pluk pěchoty\n60 důstojníků a 1294 muže, zvýšen na míru váleč-\nnou, čítati bude (včetně štábní četu pěchotní) 92\ndůstojníky a 4518 m. Jedenkaždý z 32 praporů pol-\nních myslivců v době pokoje o 15 důstojnících a\n363 mužích rozmnoží se v míře válečné na 26 dů-\nstojníků a 1205 mužů. U myslivců císařských roz-\nšíří se stav válečný na 271 důstojníků a 12.138\nmužů proti 183 důstojníkům a 3348 mužům v době\npokoje. Posádka v zemích obsazených a v Dalmacii\nskládati se bude od 1. ledna budoucího roku z 51\npraporu pěchoty, ze 6 praporů polních myslivců a\nkonečně z 1 a 10 praporů myslivců císařských.\nRakouští vojáci v Egyptě zajati. Osudná ne-\nhoda potkala několik důstojníků a námořníků ra-\nkousko-uherské dělové lodi „Neutilus.“ Rakušané ti\noctli se v zajetí egyptském v Abukiru. „Neutilus“\nplul kolem Abukiru a když spatřena byla bílá vlajka\nna baštách, domnívali se na lodi, že jest to vlajka\nparlamentární, že v Abukiru něčeho si přejí a z té\npříčiny odrazili v člunu od velké lodi jeden námořní\ndůstojník, lodní lékař dr. Kam. Theumer, námořní\nkadet, poddůstojník a 8 námořníků. Egyptský velitel\nv Abukiru pokládal patrně omylem Rakušany za\nAngličany, „Neutilus“ za anglickou loď a nedal se\nžádným ujišťováním přesvědčit. Zabral člun a zajal\nRakušany, jež kázal uvěznit v kazematech pevnostních.\nCisařská rodina zavítá do Terstu v září a po-\nbude tam od 17—19. K uvítání císaře dějí se roz-\nsáhlé přípravy a bude uspořádáno osvětlení města,\nohňostroje a osvětlení přístavu.\nMaršálek Fuad paša přijede se štábními dů-\nstojníky Nazim bejem a Zeki bejem přes Varnu do\nVídně odevzdat císaři rakouskému velký kordon řádu\nNišan Imtiaz, jejž mu sultán udělil.\nS odvodem v Bosně a Hercegovině nejde to\nFeuilleton.\nIdylla.\nNapsal Tuhošťský.\nNa úbočí horském rozkládá se malá víska, jejíž rudé\nstřechy malebně vynikají z bujné zeleně hojného stromoví.\nStříbrolesklá střecha nevysoké vížky vypíná se nad kostelí-\nkem uprostřed návsi umístěným. Na počátku vsi, u silnice,\nrozkládá se nádherná zahrada, obkličujíc starobylou zámeckou\nbudovu. Kolem zahrady vystupuje po mírném svahu úpravná\ncesta k pěknému statku, jehož vysoká lomenice svědčí, že\nnezdobí příbytek nouze. Bujná zeleň obtáčí a zakrývá stavení\naž po samou střechu, tak že jest podobno veliké besídce.\nStatný muž a žena s děckem vycházejí ze statku. Tou\ndobou otevírají se také dvéře zámecké zahrady a stará dáma\nočekává příchozí, aby se stala kmotrou mladé obča ky, která\ndnes poprvé nešena v sídlo věčného smilování .. ..\nRůžena Podhajských vyrůstala v dívku plnou něhy. Oba\nrodičové s úzkostlivou péčí dbali toho, aby žádné hrubé slovo\nnedotklo se jejího sluchu, aby žádným špatným příkladem\nnebylo útlé její srdce otupeno. Něžná kvítka rozsáhlé zahrady\na blízkých lučin a zpěvní okřídlenci byli jí nejmilejšími spo-\nlečníky hned od prvních let blahého dětství. V klidném zá-\ntiší vyrůstala skromně, a otevřela-li svá krásná modrá očka,\nzastíněná dlouhými, černými brvami, bylo lze celou její jemnou\nduši z nich vyčísti; leč sama taktéž záhy dovedla z pohledu\npoznati přání každého. Náleželať k oněm vzácným bytostem,\ns nimiž se lze dorozuměti beze slov.\nZřídka opouštěla otcův statek, jen tehda, šla-li do ko-\nposud dle přání. Při odvodu v Bileči, Stolci a Gacku\nmělo se odvésti 400 branců, ale dostavilo se jich\nvelmi málo. Vojsko prohledalo celý okres, ale schy-\ntalo jenom 150 mladíků; ostatní prchli. Oni, byvše\ndo Gacka přivedeni, zdráhali se před odvodní komisí\nudati svoje jmeno, stav, bydliště a j. Tvrdošijnější\nse stavěli na odpor, aby povolnější přiměli též k\nstejnému počínání. Bylo posláno pro oddíl pluku\nFilipoviče a hrozilo se brancům zbraní, pakli nepo-\nslechnou. Při odvodu bylo vybráno schopných pouze\n21 a ti byli odvedeni do Mostaru. Večer na to pře-\npadla četa Stojana Kovačeviče přední rakouské stráže\nu Bileče a allarmovala celou posádku. Když odstu-\npovala, poslán za ní nocí prapor myslivců, kterýž\nstřílev celou noc do větru, na úsvitě se vrátil.\nVálečná akce v Egyptě jest již v plném proudu\na byla již také anglickou vládou evropským velmo-\ncem úřadně ohlášena. Z dosavadních pohybů vojska\nanglického vysvítá zcela patrně, že chtějí Agličané\ndocíliti spojení s indickým svým vojskem, kteréž od\nSuezu postupuje a proto ustanovena Ismalie na prů-\nplavu Suezském za shromaždiště anglických vojen-\nských sborů, kteréž dílem od Port-Saidu, dílem od\nSuezu postupují a k tomu určeny jsou, aby provedly\noperaci směrem na Kahýru. Indické vojsko sestává\nze dvou brigád, sestavených ze dvou anglických a\npěti indických pluků pěchoty po 7—800 mužích, 4\nbaterií a 2 indických pluků jízdy po 550 koních,\núhrnem asi 9000 bojovníků, tak že tam bude míti\nanglický velitel celkem asi 23.000 mužů k disposici.\nProti této moci čelí u Tel-el-Kebiru v mohutných\nopevněních egyptské vojsko, čítající asi 30.000 mužů,\njemuž uloženo jest, chrániti a v moci držeti silnici\ndo Kahýry vedoucí.\nDle novějších zpráv se všeobecně připouští, že\nanglická a indická vláda nesnáze v Egyptě podce-\nňovaly. Z Anglie a Indie musejí se poslati do Egypta\nposily. Všecky středomořské posádky pošlou tam\njednotlivé volné oddíly vojska.\nWoolwichská zbrojnice dostala rozkaz, aby bez\nodkladu vypravila do Egyptu 36 oblehacích děl roz-\nličného kalibru a 1136 dělostřelců.\nDle zprávy „Golosu“ z Beyrutu jest tam ve-\nškeré obyvatelstvo bez rozdílu vyznání rozhořčeno\nstelíka, anebo ku své kmotře — do zámku. Zámek nevynikal\nničím. Obydlí nalezala se jen v přízemí a byla opatřena sta-\nromodním nábytkem; anať majitelka chtěla vše ponechati v\nten způsob, v jakém byla zažila léta dětství, přežila smrť\nobou rodičů a přála sobě také svůj život mezi známým ná-\nbytkem ukončiti. Často sedávala v zahradě s Růženou a vy-\npravovala jí upomínky ze svého mládí. A nejeden bolný vzdech\nvyloudil se děvčeti při vzpomínce, že také ona bude jednou\nsedati samotna před domkem, v němž vyrostla, aniž by více\nhlas otce neb matky k ní zazníval . . .\nŽivot na zámku byl jednotvárný, jedině o prázdninách\nnavštěvoval svou tetu sestřenec, sirotek. Tu pak Růžena\nvelmi nerada na zámek chodila, byla děcko stydlivé a neroz-\numěla žertům, kterýmiž se mladík ten rád s ní bavil.\nTak dospěla dvanáctého roku a rodiče pomýšleli na to,\naby jí poskytli příležitosti ku lepšímu vzdělání, než jakého\nse jim dostalo. Těžké bylo loučení se zámkem, trapné s rodiči.\nSlečna z Rokosů, velitelka v zámku, ještě den před od-\njezdem Růženy sama a poprvé ve svém živobití prohlížela\nveškeré jízdní náčiní, by se přesvědčila, bude-li jí možno častěji\nzajeti do města ku své milované chráněnce.\nA otec — umořil slzu v oku při myšlence, že ho nebude\nvíce Růžena vítati při návratu z pole, shrnovati vlasy s jeho\nčela a s důvěrou mu svěřovati, jakých prací za celý den\nvykonala.\nna stěhující se tam židy. V samotné Palestině proti\nním dosud nemají ničeho, protože si kupují pozemky\na začínají si hleděti rolnictví.\nCar Alexandr III. přijal na památku svého\notce titul gosudara turkestanského a nařídil, aby\ndotýčný znak přivtělen byl úplnému říšskému erbu.\nKníže černohorský nastoupil 24. srpna svoji\ncestu do Petrohradu, k níž se původně až v polovici\nzáří chystal. V jeho družině je ministr zahraničních\nzáležitostí, vojvoda Stanko Radonič, ministr vnitra,\nvojvoda Mašo Vrbica, okresní starosta ulcinský, voj-\nvoda Simo Popovič a pobočník Sako Petrovič.\nKníže černohorský stavěl se na své cestě ve\nVídni a odjel odtud včera ráno do Petrohradu. Dne\n31. srpna ráno přijal plukovníka Thömmela, rakou-\nského ministr-residenta na Cetyni, a byl návštěvou\nu arciknížete Albrechta.\nZe zadní Indie došla zpráva do zahraničního\núřadu londýnského, o vypuknutí povstání na Koree.\nUčiněn útok na královský palác, jakož i na japon-\nskou delegaci a zabiti král i královna a jistý v služ-\nbách vládních na Koree se nalezající důstojník ja-\nponský. Král rázně o to usiloval, aby připojil zem\nk zahraničnímu obchodu a tím učinil sobě nepřátele\nv řadách odpůrců připojení.\nTéž mezi Hindy a mohamedány v Salemu,\nprovinci madrasské, došlo k povážlivým náboženským\nvýtržnostem. Hindové se dopustili ukrutnosti na mo-\nhamedánech a 150 osob bylo zatčeno.\nNapnutí mezi Francií a Německem opět roste.\nSpolek německých turnéřů v Paříži, chtěl tyto dni\npořádati velkoněmeckou slavnost, což však bylo mu\npřekaženo. Podrážděnost vyvolána německými turnéry\nnabývá ve Francii rozměrů větších, než pouhé ne-\nvole proti demonstrativní velkoněmecké slavnosti\nturnéřského spolku. Od turnéřů, jejichž záměr vláda\npřekazila, přechází se k Prusku samému. Pařížské\nlisty ironisují o Německu a na vzájem činí tak listy\nněmecké o Francii. Když občanské strany se vyzý-\nvají — může z toho vzejít cos dobrého?\nI Turecko s Řeckem počíná povážlivou hru. Na\nhranicích obou tekla již krev.\nTu vcházel statkář s tváří vyjasněnou, veda sobě svou\nmilovanou Růženu, jež se právě z mésta navrátila — na po-\nsvátném místě měla poprvé opětně po čtyrech letech vstou-\npiti mezi své spoluobčany. Slečna mohla na ní oči nechati.\nOděv prostý ale čistý a vkusný zdobil ji lépe než nejdražší\nhedbáv. Rusé kadeře vroubily milostný obličej, na jehož čele\ndlela známka duševního míru, z jasněmodrých očí vyzírala\nnejryzejší nevinnosť duše.\nTak spanilou, tak něžnou a milou děvu nebyl dosud vi-\nděl — tať by královskému trůnu ku ozdobě sloužila! . .\nOdpoledne navštívila Růžena svou kmotru. Slečna chtěla\nden opětného shledání učiniti jí milým a darovala Růženě\ncyklus vybraných básní.\n— Čtete ráda? — tázal se Jindřich nesměle.\n— O dojista! — odpověděla lahodným hlasem a zar-\nděla se.\nA Jindřich usedl a předčítal dle tetina přání a co ne-\nčetl nahlas to sobě odnášela Růžena ku čtení domů. Tak\nplynula jim doba letních prázdnin v kruhu přátelském, ne-\nnuceném.\nJindřich byl se rozhodnul pro kněžský stav a očekával\njen určitou zprávu, kdy bude vysvěcen. Jen jednou ještě chtěl\na mohl se s Růženou setkati. Byl mocně rozechvěn a celé\nhodiny ztrávil v myšlenkách, bloudě po polích a lesích.\nKaždý kvítek mu připomínal její jméno, každé šelestění stromů\nvyslovovalo je. Celá příroda dýchala jen pro ni.\nCož musel navždy opustiti tu, v níž viděl jediný pramen\nsvého štěstí?\nřís\nII\n8\nisa\njím\nand\nbr\nna\nPolitický přehled.\nch\npsla\na\n*\nByla neděle a zvonek jasným zvukem volal lidstvo ku\nslovu Páně. Stará slečna vešla do kostelíka se sestřencem,\nkterý byl právě studia na universitě ukončil, a usadila se\nve starodávné lavici, v níž její předkové o bohoslužbách\n*\ndlívali.\nSeděli opětně v tiché besídce. Teta podřimovala, Jindřich\npředčítal . . . . a četl:\n"""

In [6]:
result = tst(text)
result

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 2g.20gb. Max memory: 19.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 2g.20gb. Max memory: 19.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 2g.20gb. Max memory: 19.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' Text popisuje různé náklady, včetně ceny předplatného na vydání, a uvádí, že cena pro další vydání je 33 Kč a k tomu je přidaná okamžitě placená náklad ve výši 2 Kč. Platí se předplatné každých tří měsíců a k tomu je placen náklad ve výši 1 Kč za vyslání do Ameriky a "46" Kč, které se platí každých tří měsíců. Text také uvádí silou rakouské armády, kde každý pěší pluk tvoří 60 důstojníků a 1294 mužů, a egyptské armády, s asi 30 000 muži postavenými v Tel-el-Kebir. Anglická a indická vláda podceňovala situaci v Egyptě a musí být poslány posilky z Anglie a Indie, s 36 obléhacími děla a 1136 dělníky posláni do Egypta. Text také uvádí neštěstí, při kterém se několik rakouských důstojníků a námořníků nacházelo v egyptském vězení v Abukiru.'

In [ ]:
print(result)